In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import glob
import tensorflow as tf
import zipfile

# Import scikit-learn modules for data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Import tensorflow modules for building and training the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, LSTM, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# # Specify the path to the zipped folders
# zip_file1 = "/content/drive/MyDrive/Prototype/Dataset/Foul-20230406T015240Z-001.zip"
# zip_file2 = "/content/drive/MyDrive/Prototype/Dataset/No Foul-20230406T015243Z-001.zip"

# # Extract the contents of the first zipped folder
# with zipfile.ZipFile(zip_file1, "r") as zip_ref:
#     zip_ref.extractall("/content/drive/MyDrive/Prototype/Dataset")

# # Extract the contents of the second zipped folder
# with zipfile.ZipFile(zip_file2, "r") as zip_ref:
#     zip_ref.extractall("/content/drive/MyDrive/Prototype/Dataset")

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
frame_skip = 5  # Set the number of frames to skip between extracted frames

# Define the directory containing the video data
data_dir = r"/content/drive/MyDrive/Prototype/Dataset/"

# Define the classes
classes = ["Foul", "No Foul"]

data = []
for idx, c in enumerate(classes):
    path = os.path.join(data_dir, c)
    print("Searching for videos in:", path)
    
    videos = [file for file in os.listdir(path) if file.endswith(".mp4")]
    print("Found {} videos for class {}".format(len(videos), c))
    
    for v in videos:
        video_path = os.path.join(path, v)
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_skip == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.resize(frame, (224, 224))
                data.append([frame, idx])

            frame_count += 1

        cap.release()

print("Number of frames read:", len(data))



In [ ]:
# Convert the data list to numpy arrays for input and output data
X = np.array([i[0] / 255.0 for i in data]) # Divide by 255.0 to normalize pixel values
y = np.array([i[1] for i in data])

# Print the length of the input and output data arrays
print("Length of X:", len(X))
print("Length of y:", len(y))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)
print("Train dataset class distribution:", dict(zip(unique_train, counts_train)))
print("Test dataset class distribution:", dict(zip(unique_test, counts_test)))


# Reshape X_train and X_test to (samples, time_steps, width, height, channels)
time_steps = 10  # Set the number of time steps to be used in the input
X_train = np.reshape(X_train, (-1, time_steps, 224, 224, 1))
X_test = np.reshape(X_test, (-1, time_steps, 224, 224, 1))

# Convert string labels to integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print("Classes:", label_encoder.classes_)

# Convert integer labels to one-hot encoded labels
y_train_one_hot = to_categorical(y_train_encoded)
y_test_one_hot = to_categorical(y_test_encoded)

# Reshape y_train and y_test to match the new shapes of X_train and X_test
y_train = np.reshape(y_train_one_hot, (-1, time_steps, len(classes)))
y_test = np.reshape(y_test_one_hot, (-1, time_steps, len(classes)))

# Take the first label for each sequence of frames
y_train = y_train[:, 0, :]
y_test = y_test[:, 0, :]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Define the Sequential model
model = Sequential()

# Add the layers for the model
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(time_steps, 224, 224, 1)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(256, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

# Print the summary of the model architecture
model.summary()


In [ ]:
# Compile the model with the optimizer 'adam', using 'categorical_crossentropy' as the loss function and 'accuracy' as the evaluation metric.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define EarlyStopping to stop training the model when the validation loss doesn't improve after 10 epochs.
# Also, define ModelCheckpoint to save the best model based on validation accuracy.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Define ImageDataGenerator to perform data augmentation on the training data by rescaling, applying random shear and zoom, and performing horizontal flips.
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)


# Define ImageDataGenerator to rescale the test data.
test_datagen = ImageDataGenerator(rescale=1./255)

# Calculate the number of steps for training and testing data
batch_size = 32
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

def augment_data(X, y, datagen, batch_size):
    X_reshaped = np.reshape(X, (-1, X.shape[2], X.shape[3], X.shape[4]))
    y_repeated = np.repeat(y, X.shape[1], axis=0)
    gen = datagen.flow(X_reshaped, y_repeated, batch_size=batch_size * X.shape[1])
    while True:
        X_augmented_batch, y_batch = next(gen)
        yield X_augmented_batch.reshape((-1, X.shape[1], X.shape[2], X.shape[3], X.shape[4])), y_batch[:X_augmented_batch.shape[0] // X.shape[1]]

train_generator = augment_data(X_train, y_train, train_datagen, batch_size=32)
test_generator = augment_data(X_test, y_test, test_datagen, batch_size=32)


In [ ]:
history = model.fit(train_generator, steps_per_epoch=train_steps, epochs=50, validation_data=test_generator, validation_steps=test_steps, callbacks=[es, mc])


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Save the trained model
model.save('action_rec.h5')

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_steps, verbose=1)

# Print the test set loss and accuracy
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
from sklearn.metrics import classification_report

# Get the predicted classes for the test dataset
y_test_pred = model.predict(test_generator, steps=len(X_test), verbose=1)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

# Truncate the predicted classes array to have the same length as the true classes array
y_test_pred_classes = y_test_pred_classes[:len(y_test_encoded)]

# Inverse transform the integer-encoded labels back to their original string labels
y_test_true_classes = np.array(classes)[y_test_encoded]
y_test_pred_classes = np.array(classes)[y_test_pred_classes]

# Print classification report
print(classification_report(y_test_true_classes, y_test_pred_classes, target_names=classes))
